In [1]:
%pip install --upgrade --user google-cloud-aiplatform

In [2]:
!gcloud config get-value core/project
!gcloud config set project zelarsoft-data-engineering
!gcloud auth application-default login

(unset)
Updated property [core/project].
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=BxPYKsrIWaDuXr0KGJX7Qv2Hxg52kA&prompt=consent&token_usage=remote&access_type=offline&code_challenge=lShuEiDiBDiVgUn-iL4AI3a8bvug9FAaUL669T8hsno&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJjoB_6FxDD7mOWf00eaMCNPY7z8TOnEepWrQ5lE6FEtJV1Ltpzbgux2aCPzq42Lyg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests 

In [3]:
PROJECT_ID = "zelarsoft-data-engineering"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
!pip install gradio==5.24.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.4
    Uninstalling gradio_client-1.10.4:
      Successfully uninstalled gradio_client-1.10.4
  Attempting uninstall: gradio
    Found existing installation: gradio 5.37.0
    Uninstalling gradio-5.37.0:
      Successfully uninstalled gradio-5.37.0


In [17]:
def pil_to_image(image: im) -> str:
    """Converts a PIL Image object to a base64 string."""
    with io.BytesIO() as buffer:
        image.save(buffer, format="JPEG")
        img_bytes = buffer.getvalue()
    return img_bytes

In [18]:
def resize_image(image_path, base_width=300):
  """
  Resizes an image to the specified base width while maintaining aspect ratio.

  Args:
    image_path: Path to the image.
    base_width: Desired width for the image. Defaults to 300.

  Returns:
    The resized Image object.
  """

  def calculate_new_dimensions(image_path, base_width):
    wpercent = (base_width / float(image_path.size[0]))
    hsize = int((float(image_path.size[1]) * float(wpercent)))
    return (base_width, hsize)


  img = image_path.resize(calculate_new_dimensions(image_path, base_width), im.LANCZOS)

  return img

In [19]:
def image_to_bytes(image: im) -> str:
    """Converts a PIL object to a base64 string."""
    with io.BytesIO() as buffer:
        image.save(buffer, format="JPEG")
        img_bytes = buffer.getvalue()
    return img_bytes

In [20]:
def pil_to_file(image: im, file_path: str) -> str:
    """
    Saves a PIL Image object to the specified file path.

    Args:
        image: The PIL Image object to save.
        file_path: The file path where the image will be saved.

    Returns:
        The file path where the image was saved.
    """
    try:
        image.save(file_path, format="JPEG")  # Change format if needed (e.g., PNG)
        return file_path
    except Exception as e:
        print(f"Error saving image to file: {e}")
        return None


In [21]:
import base64
import io
from io import BytesIO
from typing import Optional, Tuple, Union, List
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as im
import os
import gradio as gr
from vertexai.preview.vision_models import (
    Image,
    ImageGenerationModel,
    StyleReferenceImage,
    SubjectReferenceImage,
)
from vertexai.generative_models import GenerativeModel, Part
from vertexai.generative_models import Image as image_gen

In [45]:

# def generate_marketing_campaign(image1_path, image2_path, user_instructions, campaign_duration):
#     """Generates a marketing campaign strategy based on two images, user input, and campaign duration."""
#     try:
#         image1 = image_gen.from_bytes(image_to_bytes(image1_path))
#         image2 = image_gen.from_bytes(image_to_bytes(image2_path))

#         model = GenerativeModel("gemini-1.5-flash-002")

#         base_prompt = f"""
#         You are a brand strategist. Analyze the following two images and create a complete marketing campaign plan for a {campaign_duration} campaign.
#        1. Campaign Overview
#           Provide a compelling campaign name or tagline.
#           Define the core objective (e.g., awareness, engagement, conversions, launch).

#        2. Target Audience
#           Define the ideal audience persona(s) — age, interests, lifestyle, pain points.
#           Highlight psychographic and behavioral traits relevant to the product shown.

#       3. Channel Strategy
#           Recommend the top 3–5 platforms to focus on (e.g., Instagram, TikTok, YouTube, Pinterest, LinkedIn, Email, WhatsApp).
#           Justify each choice based on audience and content fit.

#       4. Core Messaging & Creative Themes
#           Describe the campaign’s emotional and visual narrative.
#           Include 2–3 key messages or storytelling angles to repeat throughout the campaign.
#           Highlight how the messaging connects to audience aspirations or challenges.

#        5. Content Strategy
#           List 5–8 content ideas across formats like:
#           Reels/Shorts
#           Carousel posts
#           Stories or story polls
#           Static images
#           Behind-the-scenes
#           Influencer collabs
#           User-generated content
#           Include whether content should be organic, boosted, or UGC-driven.

#        6. Tone, Style & Aesthetic
#           Define the visual identity: colors, filters, fonts, composition.
#           Specify tone of voice: e.g., aspirational, fun, educational, minimalist, luxury.

#        7. Hashtags & Emoji Strategy
#           Provide a list of 5–10 strategic hashtags (branded + niche + trending).
#           Include emoji suggestions to convey tone or boost engagement.

#        8. Timeline & Posting Calendar
#           Break down the {campaign_duration} into weeks (or days if short).
#           Suggest posting frequency and themes per week.
#           Include timing recommendations for reels vs stories vs posts.
#         """

#         full_prompt = f"{base_prompt}\n\nAdditional instructions from user: {user_instructions}"

#         prompt = [image1, image2, full_prompt]

#         response = model.generate_content(prompt)
#         return response.text.strip()
#     except Exception as e:
#         print(f"An error occurred during campaign generation: {e}")
#         return "Error generating marketing campaign."

In [46]:
# with gr.Blocks(title="AI-Powered Marketing Campaign Generator") as interface:
#     gr.Markdown("## Upload two images and generate a tailored marketing campaign plan")

#     with gr.Row():
#         image1 = gr.Image(type="pil", label="Image 1")
#         image2 = gr.Image(type="pil", label="Image 2")

#     user_instructions = gr.Textbox(
#         label="Additional Instructions",
#         placeholder="e.g., Focus on Gen Z, make it festive, include influencer strategy."
#     )

#     output = gr.Textbox(label="Generated Marketing Campaign Strategy")

#     # Buttons to choose campaign length
#     with gr.Row():
#         btn_2wk = gr.Button("Generate 2-Week Campaign")
#         btn_4wk = gr.Button("Generate 4-Week Campaign")

#     # Link buttons to the function with appropriate duration
#     btn_2wk.click(
#         fn=generate_marketing_campaign,
#         inputs=[image1, image2, user_instructions, gr.State("2-week")],
#         outputs=output
#     )

#     btn_4wk.click(
#         fn=generate_marketing_campaign,
#         inputs=[image1, image2, user_instructions, gr.State("4-week")],
#         outputs=output
#     )


# # Launch the app
# if __name__ == "__main__":
#     interface.launch(debug=True)

In [98]:
def generate_marketing_campaign(image1_path, image2_path, user_instructions, campaign_duration):
    """Generates a marketing campaign strategy based on two images, user input, and campaign duration."""
    try:
        image1 = image_gen.from_bytes(image_to_bytes(image1_path))
        image2 = image_gen.from_bytes(image_to_bytes(image2_path))

        model = GenerativeModel("gemini-1.5-flash-002")

        base_prompt = f"""
        You are a brand strategist. Analyze the following two images and create a complete marketing campaign plan for a {campaign_duration} campaign.
        Focus on:
        - Campaign objective
        - Target audience
        - Platforms to use (e.g., Instagram, YouTube, etc.)
        - Key messaging and themes
        - Content ideas (video, reels, stories, influencer posts, etc.)
        - Tone and visual style
        - Suggested hashtags and emojis must be present in a list.
        - Timeline and posting frequency over {campaign_duration}

        Combine insights from both images into a single cohesive campaign.
        """

        full_prompt = f"{base_prompt}\n\nAdditional instructions from user: {user_instructions}"

        prompt = [image1, image2, full_prompt]

        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred during campaign generation: {e}")
        return "Error generating marketing campaign."

def generate_captions_from_campaign(image1_path, image2_path, campaign_text, uploaded_csv):
    try:
        if uploaded_csv is None:
            return "Please upload a CSV file with previous captions."

        # Load captions from uploaded CSV
        df = pd.read_csv(uploaded_csv.name)
        if 'caption' not in df.columns:
            return "CSV must have a 'caption' column."

        captions = df['caption'].dropna().tolist()
        if len(captions) == 0:
            return "No captions found in the CSV."

        # Convert captions into prompt reference
        brand_examples = "\n".join([f"- {cap}" for cap in captions[:10]])  # limit to 10 examples

        image1 = image_gen.from_bytes(image_to_bytes(image1_path))
        image2 = image_gen.from_bytes(image_to_bytes(image2_path))

        model = GenerativeModel("gemini-1.5-flash-002")

        prompt = [
            image1,
            image2,
            f"""
            Based on the following marketing campaign plan:
            \"\"\"
            {campaign_text}
            \"\"\"

            Here are Raymond’s past social media captions for style reference:
            {brand_examples}

            Now generate 2 new brand-consistent social media post captions.
            - Be catchy, on-brand, and aligned with the campaign
            - Include hashtags and emojis
            - Keep tone consistent with the past captions

            Format:
            Post 1 Caption:
            ...
            Post 2 Caption:
            ...
            """
        ]

        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Error in caption generation: {e}")
        return "Error generating captions."

In [99]:
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from google.colab import userdata

In [100]:
# !pip install markdown

In [101]:
from google.colab import userdata

EMAIL = userdata.get("EMAIL")
PASSWORD = userdata.get("PASSWORD")


In [102]:
import markdown

EMAIL = userdata.get('EMAIL')
PASSWORD = userdata.get('PASSWORD')

def send_email(file_path: str, subject: str, body_markdown: str, receiver_email: str) -> str:
    """This function reads an input file and sends an email to the recipient with an attachment.
    Args:
        file_path: The path to the attachment.
        receiver_email: The receiver's email address.
        subject: The email subject.
        body: The email body.
    Returns:
        A text whether the email was sent successfully.
    """
    sender_email = EMAIL
    sender_password = PASSWORD

    if not sender_email or not sender_password:
        return "Error: Email credentials not configured."

    body_html = markdown.markdown(body_markdown)

    # Create the message.
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach plain text (fallback) and HTML
    # msg.attach(MIMEText(body_markdown, 'plain'))  # fallback
    msg.attach(MIMEText(body_html, 'html'))       # actual rendered body

    # Attach the file.
    if file_path and os.path.exists(file_path):
        try:
            with open(file_path, 'rb') as attachment:
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())
                encoders.encode_base64(part)  # Encode file in base64
                part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(file_path)}')
                msg.attach(part)
        except Exception as e:
            print(f"Error attaching file: {e}")
            return f"Error attaching file: {e}"
    elif file_path:
         print(f"Warning: Attachment file not found at {file_path}")

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
            print(f"Email sent successfully to {receiver_email}")
            return f"Email sent successfully to {receiver_email}"
    except Exception as e:
        print(f"Error: unable to send email to {receiver_email}. {e}")
        return f"Error sending email to {receiver_email}: {e}"


def handle_send_email(content, receiver_email, subject, attachment_file):
    file_path = attachment_file.name if attachment_file else None

    return send_email(file_path, subject, content, receiver_email)

In [103]:
with gr.Blocks(title="Raymond Marketing Campaign Generator") as interface:
    gr.Markdown("## Upload Two images to generate Marketing Campaign and Captions for Raymond")

    with gr.Row():
        image1 = gr.Image(type="pil", label="Image 1")
        image2 = gr.Image(type="pil", label="Image 2")

    uploaded_csv = gr.File(
        file_types=[".csv"],
        label="Upload CSV of Raymond's Previous Captions (must have a 'caption' column)"
    )

    user_instructions = gr.Textbox(
        label="Additional Instructions",
        placeholder="e.g., Make it festive and Gen Z friendly"
    )

    campaign_output = gr.Textbox(
        label="Generated Marketing Campaign Strategy",
    )


    with gr.Row():
        btn_2wk = gr.Button("Generate 2-Week Campaign")
        btn_4wk = gr.Button("Generate 4-Week Campaign")

    btn_2wk.click(
        fn=generate_marketing_campaign,
        inputs=[image1, image2, user_instructions, gr.State("2-week")],
        outputs=campaign_output
    )

    btn_4wk.click(
        fn=generate_marketing_campaign,
        inputs=[image1, image2, user_instructions, gr.State("4-week")],
        outputs=campaign_output
    )

    gr.Markdown("## Generate Social Media Captions Using Campaign + Uploaded Caption CSV")

    captions_output = gr.Textbox(
        label="Generated Social Media Captions (2 Posts)",
        lines=6
    )

    btn_captions = gr.Button("Generate Captions for Social Media Posts")

    btn_captions.click(
        fn=generate_captions_from_campaign,
        inputs=[image1, image2, campaign_output, uploaded_csv],
        outputs=captions_output
    )

    gr.Markdown("## 📤 Send Generated Campaign via Email")

    email_receiver = gr.Textbox(label="Recipient Email")
    email_subject = gr.Textbox(label="Email Subject", placeholder="e.g., Your Raymond Campaign Plan")
    email_attachment = gr.File(label="Optional Attachment", file_types=[".pdf", ".png", ".jpg"])
    email_output = gr.Textbox(label="Email Status")

    send_button = gr.Button("Send Email")

    send_button.click(
        fn=handle_send_email,
        inputs=[campaign_output, email_receiver, email_subject,email_attachment],
        outputs=email_output
    )

# Run the app
if __name__ == "__main__":
    interface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6e7bcb7d04067a5230.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Email sent successfully to lonewolf4638@gmail.com
Email sent successfully to guru.charan@zelarsoft.com
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6e7bcb7d04067a5230.gradio.live
